In [23]:
import pandas as pd
import numpy as np

sortedFeatureCorr = pd.read_csv("dataset/corr.csv")
data = pd.read_csv("dataset/preprocessed.csv")

In [24]:
#42 feature we have
first_feature_count =           21
last_feature_count =            20
depth   =   first_feature_count     +   last_feature_count  -1  -  (first_feature_count    +   last_feature_count)//5

In [25]:
a = sortedFeatureCorr.iloc[:, 0].values

In [26]:
for i in range(first_feature_count , sortedFeatureCorr.shape[0] - last_feature_count):
    data = data.drop(a[i], axis=1)
data=data.drop(list(data.columns)[0], axis=1)

In [27]:
data_npWNan = data.iloc[:, :].values
data_np = data_npWNan[data_npWNan[:,-1]!= -1 ]
unique_elements, counts_elements = np.unique(data_np[:,-1], return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     1      2      3      4      5]
 [ 78815  87257 136412 183844 275766]]


In [28]:
print("feature " + str(data_np.shape))

feature (762094, 41)


In [29]:
feature = data_np[:, :-1]
target = data_np[:, -1]

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size = 0.2, random_state = 0)


In [31]:
print(X_train.shape)
print(y_train.shape)

(609675, 40)
(609675,)


In [32]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500,
                               n_jobs=-1,
                               bootstrap = True,
                               max_depth=depth,
                               max_features = 'auto')

In [33]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=32, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [34]:
RF_predictions  =   model.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score , confusion_matrix
acc=accuracy_score(y_test,RF_predictions)
print(acc)

0.4558486802826419


In [53]:
import matplotlib.pyplot as plt
cm=confusion_matrix(y_test,RF_predictions)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.imshow(cm, interpolation='nearest',cmap="RdYlGn")
plt.title("Confusion Matrix")
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
for i in range(5):
    for j in range(5):
        plt.text(j,i,format(cm[i][j],".2f"),horizontalalignment="center",color="black")
plt.savefig("abc.pdf", bbox_inches = "tight")
plt.tight_layout()
plt.close()